## LiDAR Programs: 
Name: Divya Natekar

NYU ID: N19974330

Net ID: dyn2009

For this exercise, I was given the task of filtering the dataset through 24 trials, through any combination of the four corners in this set. From my previous program, I discovered that the best results came from the Trials 6 and 13. Hence, in this program, my goal is to produce the .ply files which map the result.

In [9]:
# ===== Build/Use six-color clouds from pass exports and harmonize palettes =====
# pip install open3d matplotlib
import os, glob, sys
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

BASE   = os.path.expanduser("~/Desktop/venv")
OUTDIR = os.path.join(BASE, "sixcolor_out")
os.makedirs(OUTDIR, exist_ok=True)

# If you already have your own six-color PLYs, put full paths here and the builder is skipped:
OVERRIDE_PRIMARY = "/Users/dnatekar82/Desktop/venv/trial06_6colors.ply"
OVERRIDE_OTHER   = "/Users/dnatekar82/Desktop/venv/trial13_6colors.ply"

POINT_SIZE = 2.0
BG_COLOR   = (0.02, 0.10, 0.16)

# --- utility ---
def load_ply(path):
    p = o3d.io.read_point_cloud(path)
    if len(p.points) == 0:
        raise ValueError(f"Empty/unreadable PLY: {path}")
    return p

def to_np(pcd):
    return np.asarray(pcd.points, float), np.asarray(pcd.colors, float) if pcd.has_colors() else None

def write_ply(path, pts, cols):
    q = o3d.geometry.PointCloud()
    q.points = o3d.utility.Vector3dVector(pts)
    q.colors = o3d.utility.Vector3dVector(cols)
    o3d.io.write_point_cloud(path, q)

def simple_render(pts, cols, out_png, point_size=2.0, bg=BG_COLOR):
    p = o3d.geometry.PointCloud()
    p.points = o3d.utility.Vector3dVector(pts)
    p.colors = o3d.utility.Vector3dVector(cols)
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False, width=2200, height=1300)
    opt = vis.get_render_option()
    opt.point_size = point_size
    opt.background_color = np.array(bg, float)
    vis.add_geometry(p)
    aabb = p.get_axis_aligned_bounding_box()
    ctr = vis.get_view_control()
    ctr.set_lookat(aabb.get_center())
    ctr.set_front([0.6,-0.5,0.6]); ctr.set_up([0,0,1]); ctr.set_zoom(0.6)
    vis.poll_events(); vis.update_renderer()
    vis.capture_screen_image(out_png, do_render=True)
    vis.destroy_window()

def legend_image(rows, title, path_png):
    fig, ax = plt.subplots(figsize=(6, max(2, 0.6*len(rows))))
    ax.axis("off")
    for i, (name, rgb, n) in enumerate(rows):
        ax.add_patch(plt.Rectangle((0.05, 0.9 - i*0.07), 0.15, 0.05, color=rgb))
        ax.text(0.23, 0.925 - i*0.07, f"{name}: {int(n):,} pts  RGB=({rgb[0]:.3f},{rgb[1]:.3f},{rgb[2]:.3f})",
                fontsize=11, va="center")
    ax.set_title(title, fontsize=13, pad=10)
    fig.tight_layout(); fig.savefig(path_png, dpi=160, bbox_inches="tight"); plt.close(fig)

def unique_palette(colors):
    key = np.round(colors, 6)
    uniq, inv = np.unique(key, axis=0, return_inverse=True)
    counts = np.bincount(inv)
    order = np.argsort(-counts)
    return uniq[order], counts[order]

def nearest_palette_indices(colors, palette):
    diff = colors[:,None,:] - palette[None,:,:]
    d2 = np.sum(diff*diff, axis=2)
    return np.argmin(d2, axis=1)

def remap_to_palette(colors, dst_palette):
    key = np.round(colors, 6)
    uniq, inv = np.unique(key, axis=0, return_inverse=True)
    idx = nearest_palette_indices(uniq, dst_palette)
    return dst_palette[idx][inv]

# --- step 1: BUILD a six-color composite from existing pass files if needed ---
def find_pass_files(base):
    # expects names like L1C1-removed.ply, L1C1-remaining.ply, … L8C4-removed/remaining.ply
    def fs(pattern):
        return sorted(glob.glob(os.path.join(base, pattern)))
    have = {f"L{L}C{C}-{kind}": fs(f"L{L}C{C}-{kind}.ply")
            for L in range(1,9) for C in range(1,5) for kind in ("removed","remaining")}
    return have

def build_six_from_passes(base, outdir):
    have = find_pass_files(base)

    # sanity: need at least L1..L4 removed + final remaining to reconstruct “not removed”
    required = [f"L{L}C{C}-removed" for L in range(1,5) for C in range(1,5)]
    missing = [k for k in required if len(have[k])==0]
    if missing:
        raise FileNotFoundError(
            "Cannot build six-color composite because these pass files are missing:\n  " +
            "\n  ".join(missing) + f"\nSearch base: {base}"
        )

    # Load all removed sets for T1 (L1..L4)
    removed_pts = []
    removed_labels = []
    # canonical palette: 4 T1 colors, 1 T2 color, 1 gray
    COL_T1 = np.array([
        [0.90,0.10,0.10],  # a
        [0.10,0.70,0.10],  # b
        [0.10,0.40,0.85],  # c
        [0.95,0.60,0.10],  # d
    ], float)
    COL_T2 = np.array([0.60,0.20,0.80], float)  # all T2 removals
    COL_KEEP = np.array([0.65,0.65,0.65], float)

    # Gather T1 removals by corner
    for L, corner_idx in zip(range(1,5), [1,2,3,4]):  # a,b,c,d
        for C in [corner_idx]:  # one per pass
            key = f"L{L}C{C}-removed"
            path = have[key][0]
            p = load_ply(path)
            pts,_ = to_np(p)
            removed_pts.append((pts, COL_T1[L-1]))
            removed_labels.append((f"T1-{chr(96+L)} (L{L}C{C})", pts.shape[0], COL_T1[L-1]))

    # Optional: T2 removals (L5..L8) if present
    t2_count = 0
    for L in range(5,9):
        any_L = [have[f"L{L}C{C}-removed"][0] for C in range(1,5) if len(have[f"L{L}C{C}-removed"])>0]
        if any_L:
            for C in range(1,5):
                key = f"L{L}C{C}-removed"
                if len(have[key])==0: continue
                p = load_ply(have[key][0])
                pts,_ = to_np(p)
                removed_pts.append((pts, COL_T2))
                t2_count += pts.shape[0]

    # Build “kept” as the last available remaining set:
    # Prefer L8 remaining, then L4 remaining
    kept_path = None
    for L in [8,4]:
        paths = []
        for C in range(1,5):
            k = f"L{L}C{C}-remaining"
            if len(have[k])>0:
                paths.append(have[k][0])
        if paths:
            # If multiple remainings exist for same layer/corner, pick the one with most points
            kept_path = max(paths, key=lambda p: len(load_ply(p).points))
            break
    if kept_path is None:
        raise FileNotFoundError("Could not find any '*-remaining.ply' to represent kept points.")

    kept_pcd = load_ply(kept_path)
    kept_pts,_ = to_np(kept_pcd)

    # Concatenate
    parts = [kept_pts]
    cols  = [np.repeat(COL_KEEP[None,:], kept_pts.shape[0], axis=0)]
    for pts,color in removed_pts:
        parts.append(pts)
        cols.append(np.repeat(color[None,:], pts.shape[0], axis=0))
    all_pts  = np.vstack(parts)
    all_cols = np.vstack(cols)

    # Save
    out_ply  = os.path.join(outdir, "sixcolors_from_passes.ply")
    write_ply(out_ply, all_pts, all_cols)
    simple_render(all_pts, all_cols, os.path.join(outdir,"sixcolors_from_passes.png"))
    rows = [(name, rgb, n) for name,n,rgb in removed_labels] + \
           [("T2 (all corners)", COL_T2, t2_count),
            ("Kept (final)", COL_KEEP, kept_pts.shape[0])]
    legend_image(rows, "Six-color composite — palette & counts", os.path.join(outdir,"palette_legend.png"))
    return out_ply

# Decide PRIMARY source
if OVERRIDE_PRIMARY and os.path.exists(OVERRIDE_PRIMARY):
    primary_path = OVERRIDE_PRIMARY
else:
    primary_path = build_six_from_passes(BASE, OUTDIR)
print("Primary six-color:", primary_path)

# Load PRIMARY and get its palette
pcdA = load_ply(primary_path)
ptsA, colA = to_np(pcdA)
palA, cntA  = unique_palette(colA)

# If you have a second six-color cloud, harmonize its colors to PRIMARY palette
if OVERRIDE_OTHER and os.path.exists(OVERRIDE_OTHER):
    pcdB = load_ply(OVERRIDE_OTHER)
    ptsB, colB = to_np(pcdB)
    colB2 = remap_to_palette(colB, palA)
    outB = os.path.join(OUTDIR, "other_harmonized_to_primary.ply")
    write_ply(outB, ptsB, colB2)
    simple_render(ptsA, colA, os.path.join(OUTDIR,"primary_render.png"))
    simple_render(ptsB, colB2, os.path.join(OUTDIR,"other_render_harmonized.png"))
    print("Other six-color harmonized →", outB)
else:
    # just render primary
    simple_render(ptsA, colA, os.path.join(OUTDIR,"primary_render.png"))

print("Outputs in:", OUTDIR)


Primary six-color: /Users/dnatekar82/Desktop/venv/trial06_6colors.ply
Other six-color harmonized → /Users/dnatekar82/Desktop/venv/sixcolor_out/other_harmonized_to_primary.ply
Outputs in: /Users/dnatekar82/Desktop/venv/sixcolor_out


### Including Ground Points:

In [13]:
# Combine a ground/retained PLY (gray) with removed classes from Trial 6 & 13.
# Outputs two PNGs: Trial6_on_ground.png and Trial13_on_ground.png

import os, numpy as np, matplotlib.pyplot as plt, open3d as o3d
from mpl_toolkits.mplot3d import Axes3D  # noqa

# ----------- EDIT THESE PATHS -----------
GROUND_PLY  = "/Users/dnatekar82/Desktop/venv/L4C4-remaining.ply"  # your ground/retained background
PLY_TRIAL6  = "/Users/dnatekar82/Desktop/venv/trial06_6colors.ply"
PLY_TRIAL13 = "/Users/dnatekar82/Desktop/venv/trial13_6colors.ply"
OUTDIR      = "/Users/dnatekar82/Desktop/venv/renders_matplotlib"
os.makedirs(OUTDIR, exist_ok=True)

# ----------- EXACT CLASS COLORS / NAMES (prof’s mapping) -----------
PALETTE = {
    "T1 – corner a":      np.array([220,  20,  60])/255.0,   # crimson
    "T1 – corner b":      np.array([ 34, 139,  34])/255.0,   # forest green
    "T1 – corner c":      np.array([ 30, 144, 255])/255.0,   # dodger blue
    "T1 – corner d":      np.array([255, 165,   0])/255.0,   # orange
    "T2 – corners a–d":   np.array([186,  85, 211])/255.0,   # orchid
    "Retained (gray)":    np.array([180, 180, 180])/255.0    # gray
}
LEGEND_ORDER = [
    "T1 – corner a", "T1 – corner b", "T1 – corner c", "T1 – corner d",
    "T2 – corners a–d", "Retained (gray)"
]
PALETTE_STACK = np.vstack([PALETTE[k] for k in LEGEND_ORDER])  # (6,3)

def snap_to_palette(rgb_floatNx3):
    # Map colors in a file to the closest of our 6 canonical colors
    diffs = rgb_floatNx3[:, None, :] - PALETTE_STACK[None, :, :]
    idx = np.argmin(np.sum(diffs*diffs, axis=2), axis=1)
    return idx  # class indices 0..5

def load_o3d_points(path):
    pcd = o3d.io.read_point_cloud(path)
    if len(pcd.points) == 0:
        raise ValueError(f"Empty or unreadable PLY: {path}")
    pts = np.asarray(pcd.points, dtype=np.float32)
    cols = np.asarray(pcd.colors, dtype=np.float32)
    return pts, cols

def set_axes_equal(ax):
    xlim, ylim, zlim = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
    xr, yr, zr = xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]
    r = 0.5*max(xr, yr, zr)
    ax.set_xlim3d(np.mean(xlim)-r, np.mean(xlim)+r)
    ax.set_ylim3d(np.mean(ylim)-r, np.mean(ylim)+r)
    ax.set_zlim3d(np.mean(zlim)-r, np.mean(zlim)+r)

def render_overlay(ground_pts, trial_pts, trial_classidx, title, outfile,
                   sample_ground=1_000_000, sample_removed=600_000,
                   s_ground=0.12, s_removed=0.18,
                   bg=(6/255,18/255,30/255), elev=18, azim=45):
    # Optional downsampling (random but deterministic)
    rng = np.random.default_rng(42)
    if ground_pts.shape[0] > sample_ground:
        gsel = rng.choice(ground_pts.shape[0], size=sample_ground, replace=False)
        ground_pts = ground_pts[gsel]
    removed_mask = trial_classidx != LEGEND_ORDER.index("Retained (gray)")
    trial_pts_removed = trial_pts[removed_mask]
    trial_cls_removed = trial_classidx[removed_mask]
    if trial_pts_removed.shape[0] > sample_removed:
        rsel = rng.choice(trial_pts_removed.shape[0], size=sample_removed, replace=False)
        trial_pts_removed = trial_pts_removed[rsel]
        trial_cls_removed = trial_cls_removed[rsel]

    fig = plt.figure(figsize=(14,8), facecolor=bg)
    ax = fig.add_subplot(111, projection='3d', facecolor=bg)
    fig.suptitle(title, color='w', fontsize=14)

    # 1) Background ground/retained in gray
    ax.scatter(ground_pts[:,0], ground_pts[:,1], ground_pts[:,2],
               s=s_ground, marker='.', lw=0, c=[PALETTE["Retained (gray)"]],
               alpha=0.95, depthshade=False)

    # 2) Overlay each removed class with its color
    handles = []
    import matplotlib.lines as mlines
    for cname in LEGEND_ORDER[:-1]:  # skip gray in overlay loop
        ci = LEGEND_ORDER.index(cname)
        mask = (trial_cls_removed == ci)
        if not np.any(mask): continue
        clr = PALETTE[cname]
        P = trial_pts_removed[mask]
        ax.scatter(P[:,0], P[:,1], P[:,2],
                   s=s_removed, marker='.', lw=0, c=[clr], depthshade=False)
        handles.append(mlines.Line2D([], [], color=clr, marker='o', linestyle='None',
                                     markersize=6, label=cname))
    # gray legend entry
    handles.append(mlines.Line2D([], [], color=PALETTE["Retained (gray)"], marker='o',
                                 linestyle='None', markersize=6, label="Retained (gray)"))

    leg = ax.legend(handles=handles, loc='upper left',
                    facecolor=(0.08,0.08,0.1), edgecolor='0.7')
    for t in leg.get_texts(): t.set_color('w')

    ax.set_xlabel('X', color='w'); ax.set_ylabel('Y', color='w'); ax.set_zlabel('Z', color='w')
    ax.tick_params(colors='0.8', labelsize=8)
    ax.view_init(elev=elev, azim=azim)
    set_axes_equal(ax)
    plt.tight_layout()
    plt.savefig(outfile, dpi=200, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.close(fig)
    print("Saved:", outfile)

# ----------- LOAD DATA -----------
g_pts, g_cols = load_o3d_points(GROUND_PLY)                   # we’ll force these to gray
t6_pts,  t6_cols  = load_o3d_points(PLY_TRIAL6)
t13_pts, t13_cols = load_o3d_points(PLY_TRIAL13)

# snap trial colors to 6-class palette, then render on top of ground
t6_classes  = snap_to_palette(t6_cols)
t13_classes = snap_to_palette(t13_cols)

render_overlay(g_pts, t6_pts,  t6_classes,
               "Trial 6 — removed classes over ground/retained",
               os.path.join(OUTDIR, "Trial6_on_ground.png"))

render_overlay(g_pts, t13_pts, t13_classes,
               "Trial 13 — removed classes over ground/retained",
               os.path.join(OUTDIR, "Trial13_on_ground.png"))

print("Done.")


Saved: /Users/dnatekar82/Desktop/venv/renders_matplotlib/Trial6_on_ground.png
Saved: /Users/dnatekar82/Desktop/venv/renders_matplotlib/Trial13_on_ground.png
Done.


### Better result for Trials 6 and 13 with Ground Points:

In [14]:
# Clean, camera-controlled renders: overlay Trial 6 / 13 removed classes on a gray ground cloud
import os, numpy as np, matplotlib.pyplot as plt, open3d as o3d

# ---- EDIT PATHS (if needed) ----
GROUND_PLY  = "/Users/dnatekar82/Desktop/venv/L4C4-remaining.ply"
PLY_TRIAL6  = "/Users/dnatekar82/Desktop/venv/trial06_6colors.ply"
PLY_TRIAL13 = "/Users/dnatekar82/Desktop/venv/trial13_6colors.ply"
OUTDIR      = "/Users/dnatekar82/Desktop/venv/renders_clean"
os.makedirs(OUTDIR, exist_ok=True)

# ---- Palette (exact, with labels) ----
PALETTE = {
    "T1 – corner a":      np.array([220,  20,  60])/255.0,   # crimson
    "T1 – corner b":      np.array([ 34, 139,  34])/255.0,   # forest green
    "T1 – corner c":      np.array([ 30, 144, 255])/255.0,   # dodger blue
    "T1 – corner d":      np.array([255, 165,   0])/255.0,   # orange
    "T2 – corners a–d":   np.array([186,  85, 211])/255.0,   # orchid
    "Retained (gray)":    np.array([180, 180, 180])/255.0    # gray
}
LEGEND_ORDER = [
    "T1 – corner a", "T1 – corner b", "T1 – corner c", "T1 – corner d",
    "T2 – corners a–d", "Retained (gray)"
]
PALETTE_STACK = np.vstack([PALETTE[k] for k in LEGEND_ORDER])  # (6,3)

def load_pts_cols(p):
    pcd = o3d.io.read_point_cloud(p)
    if len(pcd.points) == 0:
        raise ValueError(f"Empty or unreadable PLY: {p}")
    return np.asarray(pcd.points, np.float32), np.asarray(pcd.colors, np.float32)

def snap_to_palette(rgb):
    d = rgb[:,None,:] - PALETTE_STACK[None,:,:]
    return np.argmin((d*d).sum(axis=2), axis=1)  # class idx 0..5

def set_axes_equal(ax):
    # Keep aspect ratio cube-like (even when axes hidden)
    xlim, ylim, zlim = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
    xr, yr, zr = xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]
    r = 0.5*max(xr, yr, zr)
    ax.set_xlim3d(np.mean(xlim)-r, np.mean(xlim)+r)
    ax.set_ylim3d(np.mean(ylim)-r, np.mean(ylim)+r)
    ax.set_zlim3d(np.mean(zlim)-r, np.mean(zlim)+r)

def render_overlay(ground_pts, trial_pts, trial_classidx, title, outfile,
                   camera="diag",
                   sample_ground=900_000, sample_removed=700_000,
                   s_ground=0.10, s_removed=0.22,
                   bg=(6/255,18/255,30/255)):
    # camera presets
    if camera == "top":
        elev, azim = 82, -60
    elif camera == "diag":
        elev, azim = 35, -60
    else:
        elev, azim = camera  # allow tuple like (elev, azim)

    rng = np.random.default_rng(42)
    if ground_pts.shape[0] > sample_ground:
        ground_pts = ground_pts[rng.choice(ground_pts.shape[0], sample_ground, replace=False)]

    removed_mask = trial_classidx != LEGEND_ORDER.index("Retained (gray)")
    P = trial_pts[removed_mask]
    C = trial_classidx[removed_mask]
    if P.shape[0] > sample_removed:
        sel = rng.choice(P.shape[0], sample_removed, replace=False)
        P, C = P[sel], C[sel]

    # figure (no grid/axes)
    fig = plt.figure(figsize=(16,9), facecolor=bg)
    ax = fig.add_subplot(111, projection='3d', facecolor=bg)
    ax.set_axis_off()        # <- hide axes & ticks
    ax.grid(False)           # <- hide grid, just in case

    # draw gray ground
    ax.scatter(ground_pts[:,0], ground_pts[:,1], ground_pts[:,2],
               s=s_ground, marker='.', lw=0, c=[PALETTE["Retained (gray)"]],
               alpha=0.95, depthshade=False)

    # overlay removed by class color
    for cname in LEGEND_ORDER[:-1]:
        ci = LEGEND_ORDER.index(cname)
        mask = (C == ci)
        if not np.any(mask): 
            continue
        ax.scatter(P[mask,0], P[mask,1], P[mask,2],
                   s=s_removed, marker='.', lw=0, c=[PALETTE[cname]],
                   depthshade=False)

    ax.view_init(elev=elev, azim=azim)
    set_axes_equal(ax)
    plt.suptitle(title, color='w', fontsize=18)
    plt.tight_layout(pad=0)
    plt.savefig(outfile, dpi=220, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.close(fig)
    print("Saved:", outfile)

# ---- Load data once ----
g_pts, _ = load_pts_cols(GROUND_PLY)
t6_pts,  t6_cols  = load_pts_cols(PLY_TRIAL6)
t13_pts, t13_cols = load_pts_cols(PLY_TRIAL13)
t6_classes  = snap_to_palette(t6_cols)
t13_classes = snap_to_palette(t13_cols)

# ---- Make two versions for each trial ----
# Top-down (great to see scattered noise) and diagonal (3D structure)
render_overlay(g_pts, t6_pts,  t6_classes,
               "Trial 6 — removed classes over ground (TOP)", 
               os.path.join(OUTDIR,"Trial6_TOP.png"),  camera="top")

render_overlay(g_pts, t6_pts,  t6_classes,
               "Trial 6 — removed classes over ground (DIAG)",
               os.path.join(OUTDIR,"Trial6_DIAG.png"), camera="diag")

render_overlay(g_pts, t13_pts, t13_classes,
               "Trial 13 — removed classes over ground (TOP)",
               os.path.join(OUTDIR,"Trial13_TOP.png"), camera="top")

render_overlay(g_pts, t13_pts, t13_classes,
               "Trial 13 — removed classes over ground (DIAG)",
               os.path.join(OUTDIR,"Trial13_DIAG.png"), camera="diag")

print("Done. Check:", OUTDIR)


Saved: /Users/dnatekar82/Desktop/venv/renders_clean/Trial6_TOP.png
Saved: /Users/dnatekar82/Desktop/venv/renders_clean/Trial6_DIAG.png
Saved: /Users/dnatekar82/Desktop/venv/renders_clean/Trial13_TOP.png
Saved: /Users/dnatekar82/Desktop/venv/renders_clean/Trial13_DIAG.png
Done. Check: /Users/dnatekar82/Desktop/venv/renders_clean


### Trial 13 - Opposite view of dataset:

In [15]:
# Opposite diagonal view for Trial 13
render_overlay(
    g_pts, t13_pts, t13_classes,
    "Trial 13 — removed classes over ground (OPPOSITE DIAG)",
    os.path.join(OUTDIR,"Trial13_OPPOSITE_DIAG.png"),
    camera=(35, 120)   # elev=35, azim flipped to ~+120 instead of -60
)


Saved: /Users/dnatekar82/Desktop/venv/renders_clean/Trial13_OPPOSITE_DIAG.png
